# Data School : [Web scraping the President's lies in 16 lines of Python](http://www.dataschool.io/python-web-scraping-of-president-trumps-lies/#whatiswebscraping)

## Artice used : [T r u m p ’ s   L i e s](https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html)

## web scraping

This is a common scenario: You find a web page that contains data you want to analyze, but it's not presented in a format that you can easily download and read into your favorite data analysis tool. You might imagine manually copying and pasting the data into a spreadsheet, but in most cases, that is way too time consuming. A technique called web scraping is a useful way to automate this process.  

### What is web scraping? 
It's the process of extracting information from a web page __by taking advantage of patterns in the web page's underlying code__.

When converting this into a dataset, you __can think of each lie as a "record" with four fields__:

1. The date of the lie.
2. The lie itself (as a quotation).
3. The writer's brief explanation of why it was a lie.
4. The URL of an article that substantiates the claim that it was a lie.


Importantly, those fields have different formatting, which is consistent throughout the article: the date is bold red text, the lie is "regular" text, the explanation is gray italics text, and the URL is linked from the gray italics text.

Why does the formatting matter? Because it's very likely that the code underlying the web page "tags" those fields differently, and we can take advantage of that pattern when scraping the page. Let's take a look at the source code, known as HTML:

Thankfully, you only have to understand three basic facts about HTML in order to get started with web scraping :

## 1. Fact 1: HTML consists of tags

You can see that the HTML contains the article text, along with "tags" (specified using angle brackets) that "mark up" the text. ("HTML" stands for Hyper Text Markup Language.)

For example, one tag is <strong>, which means "use bold formatting". There is a _ <strong> tag before "Jan. 21" and a </strong> _ tag after it. The first is an "opening tag" and the second is a "closing tag" (denoted by the /), which indicates to the web browser where to start and stop applying the formatting. In other words, this tag tells the web browser to make the text "Jan. 21" bold. 


## 2. Fact 2: Tags can have attributes

HTML tags can have "attributes", which are specified in the opening tag. For example, <span class="short-desc"> indicates that this particular <span> tag has a class attribute with a value of short-desc.

## 3. Fact 3: Tags can be nested

# Reading the web page into Python

The first thing we need to do is to read the HTML for this article into Python, which we'll do using the [requests](http://docs.python-requests.org/en/master/) library.

In [1]:
import requests

In [2]:
r = requests.get('https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html')

The code above fetches our web page from the URL, and stores the result in a "response" object called ```r```. That response object has a ```text``` attribute, which contains the same HTML code we saw when viewing the source from our web browser:

In [3]:
# print the first 500 characters of the HTML
print(r.text[0:500])

<!DOCTYPE html>
<!--[if (gt IE 9)|!(IE)]> <!--><html lang="en" class="no-js page-interactive section-opinion page-theme-standard tone-opinion page-interactive-default limit-small layout-xlarge app-interactive" itemid="https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html" itemtype="http://schema.org/NewsArticle" itemscope xmlns:og="http://opengraphprotocol.org/schema/"><!--<![endif]-->
<!--[if IE 9]> <html lang="en" class="no-js ie9 lt-ie10 page-interactive section-opinion page


# Parsing the HTML using Beautiful Soup
We're going to parse the HTML using the Beautiful Soup 4 library, which is a popular Python library for web scraping. 

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

The code above parses the HTML (stored in r.text) into a special object called soup that the Beautiful Soup library understands. In other words, __Beautiful Soup is reading the HTML and making sense of its structure__.  

(Note that html.parser is the parser included with the Python standard library, though other parsers can be used by Beautiful Soup. See [differences between parsers](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#differences-between-parsers) to learn more.)

You might have noticed that each record has the following format:

```{html}
<span class="short-desc"><strong> DATE </strong> LIE <span class="short-truth"><a href="URL"> EXPLANATION </a></span></span>
```

There's an outer ```<span>``` tag, and then nested within it is a ```<strong>``` tag plus another 
```<span>``` tag, which itself contains an ``` <a>``` tag. All of these tags affect the formatting of the text. And because the New York Times wants each record to appear in a consistent way in your web browser, we know that each record will be tagged in a consistent way in the HTML. This is the pattern that allows us to build our dataset!

Let's ask Beautiful Soup to find all of the records:

In [5]:
results = soup.find_all('span', attrs = {'class':'short-desc'})

This code searches the soup object for all <span> tags with the attribute class="short-desc". It returns a special Beautiful Soup object (called a "ResultSet") containing the search results.

```results``` acts __like a Python list__, so we can check its length:

In [6]:
len(results)

116

There are 116 results, which seems reasonable given the length of the article. (If this number did not seem reasonable, we would examine the HTML further to determine if our assumptions about the patterns in the HTML were incorrect.)

In [7]:
results[0:3]

[<span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>,
 <span class="short-desc"><strong>Jan. 21 </strong>“A reporter for Time magazine — and I have been on their cover 14 or 15 times. I think we have the all-time record in the history of Time magazine.” <span class="short-truth"><a href="http://nation.time.com/2013/11/06/10-things-you-didnt-know-about-time/" target="_blank">(Trump was on the cover 11 times and Nixon appeared 55 times.)</a></span></span>,
 <span class="short-desc"><strong>Jan. 23 </strong>“Between 3 million and 5 million illegal votes caused me to lose the popular vote.” <span class="short-truth"><a href="https://www.nytimes.com/2017/01/23/us/politics/donald-trump-congress-democrats.html" target="_

We'll also check that the last result in this object matches the last record in the article:

In [8]:
results[-1]

<span class="short-desc"><strong>July 19 </strong>“But the F.B.I. person really reports directly to the president of the United States, which is interesting.” <span class="short-truth"><a href="https://www.usatoday.com/story/news/politics/onpolitics/2017/07/20/fbi-director-reports-justice-department-not-president/495094001/" target="_blank">(He reports directly to the attorney general.)</a></span></span>

We have now collected all 116 of the records, but we still need to separate each record into its four components (date, lie, explanation, and URL) in order to give the dataset some structure.

## Extracting the date
Web scraping is often an iterative process, in which you experiment with your code until it works exactly as you desire. To simplify the experimentation, we'll start by only working with the first record in the results object, and then later on we'll modify our code to use a loop:

In [10]:
first_result = results[0]
first_result

<span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>

Although ```first_result``` may look like a Python string, you'll notice that there are no quote marks around it. Instead, it's another special Beautiful Soup object (called a "Tag") that has specific methods and attributes.  

In order to locate the date, we can use its ```find()``` method __to find a single tag that matches a specific pattern__, in contrast to the ```find_all()``` method we used above __to find all tags that match a pattern__:

In [11]:
first_result.find('strong')

<strong>Jan. 21 </strong>


This code searches first_result for the first instance of a ```<strong>``` tag, and again returns a Beautiful Soup "Tag" object (not a string).  

Since we want __to extract the text between the opening and closing tags__, we can access its text attribute, which does in fact return a regular Python string:

In [12]:
first_result.find('strong').text

'Jan. 21\xa0'

What is ```\xa0```? You don't actually need to know this, but it's called an "escape sequence" that represents the ```&nbsp```; character we saw earlier in the HTML source.

However, you do need to know that an escape sequence represents a single character within a string. Let's slice it off from the end of the string:

In [13]:
first_result.find('strong').text[0:-1]

'Jan. 21'

Finally, we're going to add the year, since we don't want our dataset to include ambiguous dates:

In [14]:
first_result.find('strong').text[0:-1] + ', 2017'

'Jan. 21, 2017'

## Extracting the lie

In [15]:
first_result

<span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Iraq. I didn't want to go into Iraq.” <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span></span>

Our goal is to extract the two sentences about Iraq. Unfortunately, there isn't a pair of opening and closing tags that starts __immediately before the lie and ends immediately after the lie__. Therefore, we're going to have to use a different technique:

In [16]:
first_result.contents

[<strong>Jan. 21 </strong>,
 "“I wasn't a fan of Iraq. I didn't want to go into Iraq.” ",
 <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span>]

The first_result "Tag" has a contents attribute, which returns a Python list containing its "children". What are children? They are __the Tags and strings that are nested within a Tag__.

We can slice this list to extract the second element:

In [17]:
first_result.contents[1]

"“I wasn't a fan of Iraq. I didn't want to go into Iraq.” "

Finally, we'll slice off the curly quotation marks as well as the extra space at the end:

In [18]:
first_result.contents[1][1:-2]

"I wasn't a fan of Iraq. I didn't want to go into Iraq."

## Extracting the explanation

Based upon what you've seen already, you might have figured out that we have at least two options for how we extract the third component of the record, which is the writer's explanation of why the President's statement was a lie.

The __first option__ is to slice the contents attribute, like we did when extracting the lie:

In [19]:
first_result.contents[2]

<span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a></span>

The __second option__ is to search for the surrounding tag, like we did when extracting the date:

In [20]:
first_result.find('a')

<a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a>

Either way, we can access the text attribute and then slice off the opening and closing parentheses:

In [21]:
first_result.find('a').text[1:-1]

'He was for an invasion before he was against it.'

Alternatively, using the 1st option :

In [22]:
first_result.contents[2].text[1:-1]

'He was for an invasion before he was against it.'

## Extracting the URL

Finally, we want to extract the URL of the article that substantiates the writer's claim that the President was lying.

Let's examine the ```<a>``` tag within ```first_result```:

In [23]:
first_result.find('a')

<a href="https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the" target="_blank">(He was for an invasion before he was against it.)</a>

So far in this tutorial, we have been extracting text __that is between tags__. In this case, the text we want to extract is located __within the tag itself__. Specifically, we want to access the value of the ```href``` attribute within the ```<a>``` tag.  

Beautiful Soup treats tag attributes and their values like __key-value pairs in a dictionary__: you put the attribute name in brackets (like a dictionary key), and you get back the attribute's value:

In [24]:
first_result.find('a')['href']

'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the'

## Recap: Beautiful Soup methods and attributes

Before we finish building the dataset, I want to summarize a few ways you can interact with Beautiful Soup objects.

You can apply these __two methods__ to either the initial soup object or a Tag object (such as first_result):

- ```find()``` : searches for the first matching tag, and returns a Tag object
- ```find_all()```: searches for all matching tags, and returns a ResultSet object (which you can treat like a list of Tags)


You can extract information from a Tag object (such as first_result) using these __two attributes__:

- ```text```: extracts the text of a Tag, and returns a string
- ```contents```: extracts the children of a Tag, and returns a list of Tags and strings

It's important to keep track of whether you are interacting with a Tag, ResultSet, list, or string, because that affects which methods and attributes you can access.

## Building the dataset

Now that we've figured out how to extract the four components of ```first_result```, we can create a loop to repeat this process on all 116 results. We'll store the output in a __list of tuples__ called ```records```:

In [25]:
records = []
for result in results:
    date = result.find('strong').text[0:-1] + ', 2017'
    lie = result.contents[1][1:-2]
    explanation = result.find('a').text[1:-1]
    url = result.find('a')['href']
    records.append((date, lie, explanation, url))

Since there were 116 results, we should have 116 records:

In [26]:
len(records)

116

Let's do a quick spot check of the first three records:

In [27]:
records[0:3]

[('Jan. 21, 2017',
  "I wasn't a fan of Iraq. I didn't want to go into Iraq.",
  'He was for an invasion before he was against it.',
  'https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the'),
 ('Jan. 21, 2017',
  'A reporter for Time magazine — and I have been on their cover 14 or 15 times. I think we have the all-time record in the history of Time magazine.',
  'Trump was on the cover 11 times and Nixon appeared 55 times.',
  'http://nation.time.com/2013/11/06/10-things-you-didnt-know-about-time/'),
 ('Jan. 23, 2017',
  'Between 3 million and 5 million illegal votes caused me to lose the popular vote.',
  "There's no evidence of illegal voting.",
  'https://www.nytimes.com/2017/01/23/us/politics/donald-trump-congress-democrats.html')]

# Applying a tabular data structure

The last major step in this process is to apply a tabular data structure to our existing structure (which is a list of tuples). We're going to do this using the pandas library, an incredibly popular Python library for data analysis and manipulation.  

The primary data structure in pandas is the "DataFrame", which is suitable for tabular data with columns of different types, similar to an Excel spreadsheet or SQL table. We can convert our list of tuples into a DataFrame by passing it to the DataFrame constructor and specifying the desired column names:

In [28]:
import pandas as pd
df = pd.DataFrame(records, columns=['date', 'lie', 'explanation', 'url'])

In [29]:
df.head()

,date,lie,explanation,url
0,"Jan. 21, 2017",I wasn't a fan of Iraq. I didn't want to go in...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
1,"Jan. 21, 2017",A reporter for Time magazine — and I have been...,Trump was on the cover 11 times and Nixon appe...,http://nation.time.com/2013/11/06/10-things-yo...
2,"Jan. 23, 2017",Between 3 million and 5 million illegal votes ...,There's no evidence of illegal voting.,https://www.nytimes.com/2017/01/23/us/politics...
3,"Jan. 25, 2017","Now, the audience was the biggest ever. But th...",Official aerial photos show Obama's 2009 inaug...,https://www.nytimes.com/2017/01/21/us/politics...
4,"Jan. 25, 2017",Take a look at the Pew reports (which show vot...,The report never mentioned voter fraud.,https://www.nytimes.com/2017/01/24/us/politics...


The numbers on the left side of the DataFrame are known as the "index", which act as identifiers for the rows. Because we didn't specify an index, it was automatically assigned as the integers 0 to 115.

In [30]:
df.tail()

,date,lie,explanation,url
111,"July 6, 2017","As a result of this insistence, billions of do...",NATO countries agreed to meet defense spending...,http://www.nbcnews.com/politics/donald-trump/f...
112,"July 17, 2017",We’ve signed more bills — and I’m talking abou...,"Clinton, Carter, Truman, and F.D.R. had signed...",https://www.nytimes.com/2017/07/17/us/politics...
113,"July 19, 2017","Um, the Russian investigation — it’s not an in...",It is.,http://time.com/4823514/donald-trump-investiga...
114,"July 19, 2017","I heard that Harry Truman was first, and then ...","Presidents Clinton, Carter, Truman, and F.D.R....",https://www.nytimes.com/2017/07/17/us/politics...
115,"July 19, 2017",But the F.B.I. person really reports directly ...,He reports directly to the attorney general.,https://www.usatoday.com/story/news/politics/o...


Did you notice that "January" is abbreviated, while "July" is not? It's best to format your data consistently, and so we're going to convert the date column to pandas' special "datetime" format:

In [31]:
df['date'] = pd.to_datetime(df['date'])

The code above converts the "date" column to datetime format, and then overwrites the existing "date" column. (Notice that we did not have to tell pandas that the column was originally in "MONTH DAY, YEAR" format - __pandas just figured it out!)__

In [32]:
df.head()

,date,lie,explanation,url
0,2017-01-21,I wasn't a fan of Iraq. I didn't want to go in...,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-20...
1,2017-01-21,A reporter for Time magazine — and I have been...,Trump was on the cover 11 times and Nixon appe...,http://nation.time.com/2013/11/06/10-things-yo...
2,2017-01-23,Between 3 million and 5 million illegal votes ...,There's no evidence of illegal voting.,https://www.nytimes.com/2017/01/23/us/politics...
3,2017-01-25,"Now, the audience was the biggest ever. But th...",Official aerial photos show Obama's 2009 inaug...,https://www.nytimes.com/2017/01/21/us/politics...
4,2017-01-25,Take a look at the Pew reports (which show vot...,The report never mentioned voter fraud.,https://www.nytimes.com/2017/01/24/us/politics...


In [33]:
df.tail()

,date,lie,explanation,url
111,2017-07-06,"As a result of this insistence, billions of do...",NATO countries agreed to meet defense spending...,http://www.nbcnews.com/politics/donald-trump/f...
112,2017-07-17,We’ve signed more bills — and I’m talking abou...,"Clinton, Carter, Truman, and F.D.R. had signed...",https://www.nytimes.com/2017/07/17/us/politics...
113,2017-07-19,"Um, the Russian investigation — it’s not an in...",It is.,http://time.com/4823514/donald-trump-investiga...
114,2017-07-19,"I heard that Harry Truman was first, and then ...","Presidents Clinton, Carter, Truman, and F.D.R....",https://www.nytimes.com/2017/07/17/us/politics...
115,2017-07-19,But the F.B.I. person really reports directly ...,He reports directly to the attorney general.,https://www.usatoday.com/story/news/politics/o...


Not only is the date column now consistently formatted, but pandas also provides a wealth of [date-related functionality](https://pandas.pydata.org/pandas-docs/stable/timeseries.html) because it's in datetime format.

## Exporting the dataset to a CSV file
Finally, we'll use pandas to export the DataFrame to a CSV (comma-separated value) file, which is the simplest and most common way to store tabular data in a text file:

In [34]:
df.to_csv('trump_lies.csv', index=False, encoding='utf-8')

We set the index parameter to False to tell pandas that we don't need it to include the index (the integers 0 to 115) in the CSV file. You should be able to find this file in your working directory, and open it in any text editor or spreadsheet program!

In the future, you can rebuild this DataFrame by reading the CSV file back into pandas:

In [35]:
df = pd.read_csv('trump_lies.csv', parse_dates=['date'], encoding='utf-8')

 [Easier data analysis in Python with pandas](http://www.dataschool.io/easier-data-analysis-with-pandas/), or check out  [top 8 resources for learning pandas](http://www.dataschool.io/best-python-pandas-resources/).

# Summary: 16 lines of Python code
Here are the 16 lines of code that we used to scrape the web page, extract the relevant data, convert it into a tabular dataset, and export it to a CSV file:

```

import requests
r = requests.get('https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html')

from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all('span', attrs={'class':'short-desc'})

records = []
for result in results:
    date = result.find('strong').text[0:-1] + ', 2017'
    lie = result.contents[1][1:-2]
    explanation = result.find('a').text[1:-1]
    url = result.find('a')['href']
    records.append((date, lie, explanation, url))

import pandas as pd
df = pd.DataFrame(records, columns=['date', 'lie', 'explanation', 'url'])
df['date'] = pd.to_datetime(df['date'])
df.to_csv('trump_lies.csv', index=False, encoding='utf-8')


```